# MultiThreading: The Simple Monitor of Computer


When you start a program, the operating system creates a new **process(进程)** in which the program is executed. 

A process consists of one or more **threads(线程)**. Each thread is a partial process that executes a sequence of instructions **independently** of other parts of the process.

Within a process or program, we can run multiple threads concurrently to improve the performance.

For example, while one thread is blocked (e.g., waiting for completion of an I/O operation), another thread can use the CPU time to perform computations, resulted in better performance and overall throughput.



## 1 Unresponsive UI  


### 1.1 The Application with blocking IO

**Blocking** allows functions to maintain **synchronization(同步) and honor the intended flow of execution** through the code. The action or data that the call is requesting may or may **not be available** at the time the call is made, so a `blocking` call will `wait` for the other end to respond in some fashion before returning to the caller. As a side effect, it will also effectively `suspend your application` until it returns.

**psutil**  https://github.com/giampaolo/psutil

* Cross-platform lib for process and system monitoring in Python 

In [ ]:
import time
import psutil
import datetime

def io_cpu_percent():
    time.sleep(2)
    return psutil.cpu_percent()

def io_mem_percent():
    time.sleep(2)
    return psutil.virtual_memory().percent

def io_sensors_battery_percent():
    time.sleep(2)
    return psutil.sensors_battery().percent

tagfuncs={"CPU_PERCEN": io_cpu_percent,
           "MEM_PERCENT": io_mem_percent,
           "BAT_PERCENT": io_sensors_battery_percent
          }

print("Begin of Submitting the monitorings")
for tag in tagfuncs.keys():
    str_curtime=datetime.datetime.now().strftime('%H:%M:%S')
    print('\n\tBegin of {} at {}'.format(tag,str_curtime))
    value=tagfuncs[tag]()
    print("\t",tag,value)
    str_curtime=datetime.datetime.now().strftime('%H:%M:%S')
    print('\tEnd of {} at {}'.format(tag,str_curtime))

print("\nEnd of Submitting he monitorings")

In the following diagram, the three `tasks` are represented as `boxes`. The time spent by the  <b style="color:orange">CPU processing and submitting the request</b> is in  <b style="color:orange">orange</b>  while the  <b style="color:blue">waiting</b> times are in <b style="color:blue">blue</b>. You can see how most of the time is spent waiting for the resources while our machine sits
idle without doing anything else:

![three_tasks](./img/three_tasks.jpg)

### 1.2 The Unresponsive Monitor 

**Deques(双向队列)**
https://docs.python.org/3.7/library/collections.html#collections.deque

Deques are a generalization of stacks and queues (the name is pronounced “deck” and is short for “**double-ended queue**”). Deques support thread-safe, memory efficient appends and pops from **either side of the deque** with approximately the same O(1) performance in either direction.

```python
from collections import deque

```
**matplotlib.animation.FuncAnimation**

https://matplotlib.org/api/_as_gen/matplotlib.animation.FuncAnimation.html#matplotlib.animation.FuncAnimation

Makes an animation by repeatedly calling a function **func**.

```python
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
```
**matplotlib.pyplot.table**

https://matplotlib.org/api/_as_gen/matplotlib.pyplot.table.html

Add a table to the current axes.


**Test**

Set `interval=1000`
```python
ani = FuncAnimation(fig, update,init_func=init, blit=True,interval=1000)
```

1. The times of Blocking IO > the `interval=1000`

```python
time.sleep(2)
```

2. The times of Blocking IO < the `interval=1000`

```python
time.sleep(0.1)
```    

In [ ]:
%%file ./code/concurrency/monitor_unresponsive.py
import time
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import psutil

def GetTagData(tag):
    time.sleep(2)
    tagfuncs={"CPU_PERCENT": psutil.cpu_percent(),
               "MEM_PERCENT": psutil.virtual_memory().percent,
               "BAT_PERCENT": psutil.sensors_battery().percent} 
    try:              
        value= tagfuncs[tag]
        rc=1    
    except:
        rc,value=0,None  
    return (rc,value)        
            
tag="CPU_PERCENT"
y = deque()

columns = ()
col_labels = ['Tag', 'Unit', 'Value']
table_vals = [[tag,"%",""]]


fig, ax = plt.subplots()
ax.set_title("The Simple Monitor:"+tag)
ln, = plt.plot([], [], 'b-o')
str_cursecond=str(time.localtime(time.time()).tm_sec)   
time_text = ax.text(0.5, 80, "")

tbl = ax.table(cellText=table_vals,
               colLabels=col_labels,
               colWidths=[0.2] * 3,
               cellLoc='center',
               loc='best')

def init():
    ax.set_xlim(0, 9)
    ax.set_ylim(0, 100)
    return ln,

def update(frames):
    # blocking io -> unresponsive_monitor
    rc,value =GetTagData(tag)
    if len(y) < 10:
        y.append(value)
    else:
        y.popleft()
        y.append(value)

    str_curtime=time.strftime("%F %H:%M:%S", time.localtime(time.time()))
    time_text.set_text("Time:"+str_curtime)
    
    table_vals = [[tag,"%",str(value)]]
    tbl = ax.table(cellText=table_vals,
               colLabels=col_labels,
               colWidths=[0.2] *3,
               cellLoc='center',
               loc='best')

    ln.set_xdata(np.arange(len(y)))
    ln.set_ydata(np.array(y))
    return ln,time_text, tbl

ani = FuncAnimation(fig, update,init_func=init, blit=True,interval=1000)
plt.show()

## 2 Responsive UI  

We’ll look at the way to realize **Responsive UI** using **Multithreading**.

### 2.1  Threading 

In Python，the threading module provides APIs for managing several threads of execution, which allows a program to run multiple operations **concurrently** in the same process space.

* [threading — Thread-based parallelism](https://docs.python.org/3/library/threading.html)


#### 2.1.1 Create a thread

The simplest way to use a **Thread** is to instantiate it with a **target** function and call **start()** to let it begin working


In [ ]:
import threading
import time
def worker():
    """thread worker function"""
    time.sleep(1)
    print('Worker\n')

print(threading.current_thread().name+" Begin!")
for i in range(5):
    t = threading.Thread(target=worker)
    t.start()
print(threading.current_thread().name+" Exit!")

#### 2.1.2 join() Method

```python
join(timeout=None)
```
Wait until the thread terminates. This **blocks** the calling thread until the thread whose `join()` method is called terminates

* either normally or through an unhandled exception 

* until the optional timeout occurs.



In [ ]:
import threading
import time
import logging

def worker():
    print('Starting worker')
    time.sleep(3)
    print('Exiting worker')

start_time = time.time()
d = threading.Thread(target=worker)
d.start()
d.join()
print(threading.current_thread().name+" Exiting!")
print('Total Time：', time.time()-start_time)

### 2.2 Threading IO  

In [ ]:
import threading
import time
import psutil
import datetime

def GetTagData(tag):
    time.sleep(2)
    tagfuncs={"CPU_PERCENT": psutil.cpu_percent(),
                   "MEM_PERCENT": psutil.virtual_memory().percent,
                   "BAT_PERCENT": psutil.sensors_battery().percent} 
    try:              
        value= tagfuncs[tag]
        rc=1    
    except:
        rc,value=0,None  
    return (rc,value)        

def io_worker(tag):
    """thread worker function"""
    str_curtime=datetime.datetime.now().strftime('%H:%M:%S.%f')
    print('\n\tBegin of {} at {}'.format(tag,str_curtime),end="")
    rcvalue=GetTagData(tag)
    print("\n\n\t",tag,rcvalue)
    str_curtime=datetime.datetime.now().strftime('%H:%M:%S.%f')
    print('\tEnd of {} at {}'.format(tag,str_curtime),end="")

tags=["CPU_PERCENT","MEM_PERCENT","BAT_PERCENT"]    
print("Begin of Submitting tags")
for tag in tags:
    t = threading.Thread(target=io_worker,args=(tag,))
    t.start()
print("\n\nEnd of Submitted  tags")    

In the following figure, you can see that as soon as we submit our request `virtual_interface_data(tag1)` in one worker thread, we can start preparing for next  worker thread of  `virtual_interface_data(tag2)` and so on. 

This allows us to reduce the CPU waiting time and to start processing the results as soon as they become available:

![three_tasks_concurrency](./img/three_tasks_concurrency.jpg)

>The time spent by the  <b style="color:orange">CPU processing and submitting the request</b> is in  <b style="color:orange">orange</b>  while the  <b style="color:blue">waiting</b> times are in <b style="color:blue">blue</b>.


### 2.3 Exchange data using Queue

It is often encountered when working with MultiThreads: safely **communicate or exchange data** between them.

One such interaction is the **producer/consumer** relationship. 

Perhaps the safest way to send data from one thread to another is to use a **Queue** from the **queue** library. To do this, you create a Queue instance that is shared by the threads.

Threads then use `put()` or `get(`) operations to add or remove items from the `queue`. 

>The principal challenge of multi-threaded applications is coordinating threads that **share data or other resources**. To that end, the threading module provides a number of synchronization primitives including locks, events, condition variables, and semaphores.
While those tools are powerful, minor design errors can result in problems that are difficult to reproduce.
>
>So, the preferred approach to task coordination is to concentrate all access to a resource in a single thread
and then use the queue module to feed that thread with requests from other threads. Applications using
Queue objects for inter-thread communication and coordination are easier to design, more readable, and
more reliable.

#### 2.3.1 Queue

**Queue** instances already have all of the required `locking`, so they can be safely shared by as many threads as you wish.

>[queue — A synchronized queue class](https://docs.python.org/3/library/queue.html)
>The queue module implements multi-producer, multi-consumer queues. It is especially useful in threaded programming when information must be exchanged safely between multiple threads

When using `queues`, it can be somewhat tricky to coordinate the shutdown of the producer and consumer. A common solution to this problem is to rely on a special sentinel value, which when placed in the queue, causes consumers to terminate.



#### 2.3.2 Timer Objects


This class represents an action that should be run only after a certain amount of time has passed — a timer. `Timer` is a subclass of `Thread` and as such also functions as an example of creating custom threads.

Timers are started, as with threads, by calling their `start(`) method. The timer can be stopped (before its action has begun) by calling the `cancel()` method. The interval the timer will wait before executing its action may not be exactly the same as the interval specified by the user.

We can initialize a `threading.Timer` instance by passing the amount of time we want to `wait and a callback`. 

A `callback` is simply a function that will be called when the timer expires. Note that we have to also call the `Timer.start` method to activate the timer:

In [ ]:
from queue import Queue
from threading import Thread,Timer
import time
import sys  

def GetTagData(tag):
    time.sleep(2)
    tagfuncs={"CPU_PERCENT": psutil.cpu_percent(),
              "MEM_PERCENT": psutil.virtual_memory().percent,
              "BAT_PERCENT": psutil.sensors_battery().percent} 
    try:              
        value= tagfuncs[tag]
        rc=1    
    except:
        rc,value=0,None  
    return (rc,value)   

def PeriodDataProducer(delay,tag,out_q):
    rc,value= GetTagData(tag)
    if rc == 1:
        out_q.put((rc,value))
    else:
        pass
    t=Timer(delay, PeriodDataProducer,(delay,tag,out_q))
    t.start()

def DataConsumer(in_q):
    while True:
        rcvalue = in_q.get()
        print(rcvalue[0],rcvalue[1])
        
q = Queue()
tag="BAT_PERCENT"
delay=2
p=PeriodDataProducer(delay,tag,q)
c= Thread(target= DataConsumer, args=(q,)) 
c.start()
q.join()

### 2.4 The Responsive Monitor 

In [ ]:
%%file ./code/concurrency/monitor_multithreading.py
from threading import Thread, Timer
from queue import Queue
import time
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import psutil

def GetTagData(tag):
    time.sleep(2)
    tagfuncs={"CPU_PERCENT": psutil.cpu_percent(),
               "MEM_PERCENT": psutil.virtual_memory().percent,
               "BAT_PERCENT": psutil.sensors_battery().percent} 
    try:              
        value= tagfuncs[tag]
        rc=1    
    except Exception as msg:
        rc,value=0,None  
    return (rc,value)  

def PeriodDataProducer(delay,tag,out_q):
    rc, value =  GetTagData(tag)
    if rc == 1:
        out_q.put((rc, value))
    else:
        out_q.put((rc, value))
    t = Timer(delay, PeriodDataProducer, (delay, tag,out_q))
    t.start()


def DataConsumerPlot(in_q,npoints):
    y = deque()
    plt.figure()
    plt.title("The Simple CPU Percent Monitor")
    lines, = plt.plot([], [], "b-o")
    time_text = plt.text(0.5, 80, "")
    plt.xlim(0, npoints-1)
    plt.ylim(0, 100)
    
    columns = ()
    col_labels = ['Tag', 'Unit', 'Value']
    table_vals = [[tag,"%",""]]
    tbl = plt.table(cellText=table_vals,
               colLabels=col_labels,
               colWidths=[0.2] * 3,
               cellLoc='center',
               loc='best')

    def DataConsumer(in_q,npoints):
        while True:
            rcvalue = in_q.get()
            if len(y) < npoints:
                y.append(rcvalue[1])
            else:
                y.popleft()
                y.append(rcvalue[1])

            lines.set_xdata(np.arange(len(y)))
            lines.set_ydata(np.array(y))
            str_curtime=time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))   
            time_text.set_text("Time:"+str_curtime)
            
            table_vals = [[tag,"%",str(np.array(y)[-1])]]
            tbl = plt.table(cellText=table_vals,
                  colLabels=col_labels,
                  colWidths=[0.2] * 3,
                  cellLoc='center',
                  loc='best')
        
            plt.draw()

    c = Thread(target=DataConsumer, args=(in_q,npoints))
    c.start()
    plt.show()


if __name__ == '__main__':
    q = Queue()
    delay = 3
    tag="CPU_PERCENT"
    p = PeriodDataProducer(delay,tag, q)
    npoints = 10
    DataConsumerPlot(q,npoints)
    q.join()

## 3 Reactive programming

**Reactive programming(响应式编程)** is a paradigm that aims at building better concurrent systems. Reactive applications are designed to comply with the requirements exemplified by the reactive manifesto:

* **Responsive**: The system responds **immediately** to the user.

* **Elastic**: The system is capable of handling different levels of load and is able to adapt to accommodate increasing demands.

* **Resilient**: The system deals with failure gracefully. This is achieved by modularity and avoiding having a single point of failure.

* **Message driven**: The system should not block and take advantage of `events` and `messages`. A message-driven application helps achieve all the previous requirements.

As you can see, the intent of reactive systems is quite noble, but how exactly does reactiveprogramming work? In this section, we will learn about the principles of reactive programming using the **RxPy** library.

```
python -m pip install rx
```

>The RxPy library is part of [ReactiveX](http://reactivex.io), which is a project that implements reactive programming tools for a large variety of languages.



In [ ]:
%%file  ./code/concurrency/monitor_reactive.py
from rx import Observable
import numpy as np
import matplotlib.pyplot as plt
import time
import psutil

def GetTagData(tag):
    time.sleep(0.2)
    tagfuncs={"CPU_PERCENT": psutil.cpu_percent(),
               "MEM_PERCENT": psutil.virtual_memory().percent,
               "BAT_PERCENT": psutil.sensors_battery().percent} 
    try:              
        value= tagfuncs[tag]
        rc=1    
    except Exception as msg:
        rc,value=0,None  
    return (rc,value)  

intervalTime=100
delay=1
tag="CPU_PERCENT"

cpu_data = (Observable
            .interval(intervalTime) 
            .map(lambda rc,value: GetTagData(tag))
            .publish())

cpu_data.connect()

def monitor_cpu(npoints):
    plt.figure()
    plt.title("The Simple CPU Percent Monitor")
    lines, = plt.plot([], [],"b-o")
    time_text = plt.text(0.5, 80, "")
    plt.xlim(0, npoints-1)
    plt.ylim(0, 100)
    
    columns = ()
    col_labels = ['Tag', 'Unit', 'Value']
    table_vals = [[tag,"%",""]]
    
    tbl = plt.table(cellText=table_vals,
               colLabels=col_labels,
               colWidths=[0.2] * 3,
               cellLoc='center',
               loc='best')

    cpu_data_window = cpu_data.buffer_with_count(npoints, 1)
    
    def update_plot(cpu_readings):
        lines.set_xdata(np.arange(len(cpu_readings)))
        lines.set_ydata(np.array(cpu_readings)[:,1])
        str_curtime=time.strftime("%F %H:%M:%S", time.localtime(time.time()))  
        
        time_text.set_text("Time:"+str_curtime)
        
        table_vals = [[tag,"%",str(np.array(cpu_readings)[:,1][-1])]]
        tbl = plt.table(cellText=table_vals,
               colLabels=col_labels,
               colWidths=[0.2] * 3,
               cellLoc='center',
               loc='best')
         
        plt.draw()
    
    cpu_data_window.subscribe(update_plot)
    plt.show()

if __name__ == '__main__':
    monitor_cpu(10)

## Reference

Allen B. Downey. [Think OS:A Brief Introduction to Operating Systems](http://greenteapress.com/wp/think-os/)

[The Python Standard Library:Concurrent Execution](https://docs.python.org/3/library/concurrency.html)

* [threading — Thread-based parallelism](https://docs.python.org/3/library/threading.html)

* [queue — A synchronized queue class](https://docs.python.org/3/library/queue.html)

Doug Hellmann. The Python3 Standard Library by Example,Pearson Education, Inc. 2017

* http://doughellmann.com/blog/the-python-3-standard-library-by-example

* Python 3 Module of the Week https://pymotw.com/3/
   
Gabriele Lanaro. Python High Performance,Second Edition, Packt Publishing,2017
